In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
import streamlit as st
import pickle

In [10]:
data = pd.read_excel('itog3.xlsx')

In [11]:
data.shape

(6399, 3)

In [12]:
data.head (5)

,text,tags,topic
0,МАЙЕР ГАЛИНА ИВАНОВ НА Г Р ПРОЖ УЛ Л ...,330,Самоуправство
1,Г В ЧАС НА УЛ ОСТРОУХОВА НЕИЗВЕСТНЫЙ ...,115 Ч.2,Умышленное причинение легкого вреда здоровью
2,Г В НА МОСКОВСКОМ ПР-ТЕ НЕИЗВЕСТНЫЙ ПРИЧИ...,115 Ч.2,Умышленное причинение легкого вреда здоровью
3,Г В НА УЛ ЗАПОЛЯРНАЯ А НЕИЗВЕСТНЫЙ МОШЕ...,159 Ч.2,Мошенничество
4,Г В НА УЛ ПОДКЛЕТНЕНСКАЯ НЕИЗВЕСТНЫЙ ИЗ Х...,116 Ч.2,Побои


In [13]:
print(data['topic'].unique(), len(data['topic'].unique()))

['Самоуправство' 'Умышленное причинение легкого вреда здоровью'
 'Мошенничество' 'Побои' 'Кража'
 'Уклонение от отбывания ограничения свободы'
 'Нецелевое расходование средств государственных внебюджетных фондов.'
 'Незаконное образование юридического лица'
 'Незаконный оборот наркотических средств'
 'Умышленное причинение тяжкого вреда здоровью' 'Угон'
 'Угроза убийством или причинением тяжкого вреда здоровью'
 'УСТАНОВЛЕНО МЕСТОНАХОЖДЕНИЕ' 'ЗАДЕРЖАН НАХОДИВШИЙСЯ В РОЗЫСКЕ' 'Грабеж'
 'ОБНАРУЖЕНИЕ ТРУПА' 'Изнасилование' 'Убийство' 'САМОУБИЙСТВО'
 'Заведомо ложное сообщение об акте терроризма' 'ДТП'
 'Неправомерное завладение автомобилем или иным транспортным средством без цели хищения'
 'Изготовление, хранение, перевозка или сбыт поддельных денег или ценных бумаг'
 'Умышленное причинение средней тяжести вреда здоровью'
 'Применеие насилия в отношении представителя власти'
 'БЕЗ ВЕСТИ ПРОПАВШИЕ'
 'Неисполнение приговора суда, решения суда или иного судебного акта'
 'НЕСЧАCТНЫЙ СЛУЧАЙ' '

In [134]:
topics = ['Грабеж', 'Разбой',  'Кража', 'Мошенничество', 'Незаконный оборот наркотических средств',  'Умышленное причинение средней тяжести вреда здоровью','Убийство', 'Изготовление, хранение, перевозка или сбыт поддельных денег или ценных бумаг' ,'Умышленное причинение тяжкого вреда здоровью' ]

In [135]:
per=500

In [136]:
 df_res = pd.DataFrame()

for topic in tqdm(topics):
        df_topic = data[data['topic'] == topic][:per]
        
        df_res = df_res.append(df_topic, ignore_index=True)

     

In [138]:
df_res.shape

(2823, 3)

In [139]:
df_res['topic'].value_counts()

Грабеж                                                                          500
Мошенничество                                                                   500
Незаконный оборот наркотических средств                                         500
Кража                                                                           500
Изготовление, хранение, перевозка или сбыт поддельных денег или ценных бумаг    261
Умышленное причинение тяжкого вреда здоровью                                    219
Разбой                                                                          151
Умышленное причинение средней тяжести вреда здоровью                            115
Убийство                                                                         77
Name: topic, dtype: int64

In [140]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)
import nltk
nltk.download("stopwords")
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MIGRAT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['text'])]

In [142]:
len(prep_text)
prep_text[0]

' в на ул беговой д в торговом зале магазина пятерочка неизвестный применив насилие не опасное для жизни и здоровья открыто похитил продукты питания прин ооо агроторг '

In [143]:
df_res['text_prep'] = prep_text

In [144]:
df_res.head(1)

,text,tags,topic,text_prep
0,"21.09.2018 В 14.05 НА УЛ. БЕГОВОЙ, Д. 8/1, В ...",161 Ч.2,Грабеж,в на ул беговой д в торговом зале магазина пя...


In [145]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian")

In [146]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

In [147]:
from nltk import word_tokenize

stemmed_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_res['text_stem'] = stemmed_texts_list

In [148]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MIGRAT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [149]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [150]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_res['text_sw'] = sw_texts_list

In [151]:
df_res['text_sw'][0]

'ул беговой торговом зале магазина пятерочка неизвестный применив насилие опасное жизни здоровья открыто похитил продукты питания прин ооо агроторг'

In [152]:
df_res.to_csv('kusp_stemmed.csv')

In [153]:
df_res['text_stem'][0]

'ул бегов торгов зал магазин пятерочк неизвестн примен насил опасн жизн здоров открыт похит продукт питан прин оо агроторг'

In [154]:
X = df_res['text_sw']
y = df_res['topic']

In [155]:
type(df_res['text_sw'][0])

str

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [157]:
my_tags = df_res['topic'].unique()
my_tags

array(['Грабеж', 'Разбой', 'Кража', 'Мошенничество',
       'Незаконный оборот наркотических средств',
       'Умышленное причинение средней тяжести вреда здоровью', 'Убийство',
       'Изготовление, хранение, перевозка или сбыт поддельных денег или ценных бумаг',
       'Умышленное причинение тяжкого вреда здоровью'], dtype=object)

In [158]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [159]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [160]:
%%time
nb.fit(X_train, y_train)

Wall time: 132 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [161]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

Wall time: 49.8 ms


In [162]:
y_pred[0]

'Кража'

In [163]:
print(X_test[0], y_test[0], y_pred[0])

KeyError: 0

In [164]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [165]:
%%time
logreg.fit(X_train, y_train)

Wall time: 1.59 s


C:\Users\MIGRAT\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [166]:
%%time
y_pred = logreg.predict(X_test)


Wall time: 49.3 ms


In [167]:
from sklearn.metrics import accuracy_score
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9090909090909091
                                                                              precision    recall  f1-score   support

                                                                      Грабеж       0.89      0.93      0.91       155
                                                                      Разбой       0.96      0.99      0.97        68
                                                                       Кража       0.97      0.94      0.95       149
                                                               Мошенничество       0.96      0.98      0.97       161
                                     Незаконный оборот наркотических средств       0.99      0.99      0.99       145
                        Умышленное причинение средней тяжести вреда здоровью       0.74      0.65      0.69        48
                                                                    Убийство       0.63      0.46      0.53        26
Изготовление, хранение, пер

In [171]:
econ_text = '''
 найден мужчина без сознания со следами множественных ножевых ранений в грудь
'''

In [45]:
econ_text = st.text_input('Введите текст')
st.button('Выполнить')

2021-04-14 18:49:11.386 WARNING root: 
  command:

    streamlit run C:\Users\MIGRAT\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


False

In [169]:
%%time
econ_text = lemmatize_text(econ_text)

Wall time: 7.06 s


In [172]:
ect_pred = logreg.predict([econ_text])
ect_pred

array(['Убийство'], dtype=object)

In [173]:
filename = 'finalized_model2.pkl'
pickle.dump(logreg, open(filename, 'wb'))

loaded_model = pickle.load(open('finalized_model2.pkl', 'rb'))
result = loaded_model.score(X_test, y_test)
result

0.9090909090909091